<a href="https://colab.research.google.com/github/averma12/Grpo_experiments/blob/main/Qwen_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

In [2]:
!pip install git+https://github.com/huggingface/trl.git

  Cloning https://github.com/huggingface/trl.git to /tmp/pip-req-build-5da0ufk6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-5da0ufk6
  Resolved https://github.com/huggingface/trl.git to commit 69ad852e5654a77f1695eb4c608906fe0c7e8624
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.16.0.dev0-py3-none-any.whl size=322988 sha256=a4e070ac2c01a91b7a6e7afda8b9aef0486b5c11ae4993d3332d92af1d4be7ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-xmfefq6o/wheels/e7/e5/ec/8cce76372e10e954b47f3893a36ff7c0786d9dbc846efa8860
Successfully built trl
  Attempting uninstall: trl
    Found existing installation: trl 0.15.0.dev0
    Uninstalling trl-0.15.0.dev0:
      Successfully uninstalled trl-0.15.0.dev0


In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 16384 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 02-25 18:57:45 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.37%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 16384. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 21.18 GB. Also swap space = 6 GB.
INFO 02-25 18:58:04 config.py:549] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'e

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [4]:
import re
from datasets import load_dataset, Dataset,concatenate_datasets

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()
def get_openthoughts_questions(split = "train") -> Dataset:
    data = load_dataset("open-thoughts/OpenThoughts-114k","default")["train"]
    data_map = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['conversations'][0]["value"]}
        ],
        'answer': (x['conversations'][1]["value"])
    })
    return data_map
additional_data = get_openthoughts_questions()
dataset = concatenate_datasets([dataset, additional_data])

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

...

train-00003-of-00006.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/154M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/113957 [00:00<?, ? examples/s]

Map:   0%|          | 0/113957 [00:00<?, ? examples/s]

In [5]:
from trl import GRPOConfig, GRPOTrainer

#template = AutoTemplate.from_pretrained("google/flan-t5-xl")  # or any suitable template
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [6]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 121,430 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 250
 "-____-"     Number of trainable parameters = 59,867,136


-------------------- Question:
Return your final response within \boxed{}. Construct a circle given the lengths of the tangents $(a, b, c)$ drawn from three given points to the circle. 
Answer:
<|begin_of_thought|>

Okay, so I need to figure out how to construct a circle when given the lengths of the tangents from three points. Let me start by recalling some geometry concepts. 

First off, if we have a point outside a circle, the length of the tangent from that point to the circle is given by the formula \( \sqrt{(x - h)^2 + (y - k)^2 - r^2} \), where (h, k) is the center of the circle and r is the radius, right? So, if we have three points with known tangent lengths, we can set up equations based on this formula. 

Let me denote the three given points as A, B, and C, and their respective tangent lengths as a, b, and c. The coordinates of these points might be needed, but the problem doesn't specify if the points are given in a coordinate system or if we need to construct this in a gen

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.125000,0.000000,200.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.125000,0.000000,200.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.533125,0.084155,200.000000,0.000000,-0.533125,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.117125,0.448800,200.000000,0.001077,-0.117125,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.042875,0.125962,162.125000,0.000443,-0.042875,0.000000,0.000000,0.000000,0.000000
6,0.000300,-0.537500,0.027542,200.000000,0.006955,-0.537500,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.093750,0.057864,200.000000,0.000394,0.093750,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.022375,0.193622,197.875000,0.000493,0.022375,0.000000,0.000000,0.000000,0.000000
9,0.000100,0.057500,0.190919,200.000000,0.001662,0.057500,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.125000,0.000000,200.000000,0.000326,0.125000,0.000000,0.000000,0.000000,0.000000


Streaming output truncated to the last 5000 lines.

Because when a girl takes floor(C/n) candies, the remaining candies adjust in such a way that the next girl also takes floor(C/n), even though the number of children has decreased. This seems counterintuitive, but in the examples, it worked.

But how to formally prove this?

Let’s consider the total taken by girls. If each girl takes floor(C/n), then the total taken by girls would be g * floor(C/n). The remaining candies would be C - g * floor(C/n). Then the boys take the remaining candies, which is C - g * floor(C/n). Since there are b boys, they would take ceil((C - g * floor(C/n))/b). But wait, this doesn't necessarily hold. For example, in the fourth example, C=10, n=4, g=2, floor(C/n)=2. Remaining for boys:10 - 2*2=6. Number of boys=2. 6/2=3. Boys take 3 each, total 6. Which matches.

In the second example, C=1000, n=3, g=1, floor(C/n)=333. Remaining for boys:1000 -333=667. Number of boys=2. 667/2=333.5. Boys take 334 and 333, to

TrainOutput(global_step=250, training_loss=0.0012243206474032604, metrics={'train_runtime': 1599.8875, 'train_samples_per_second': 1.25, 'train_steps_per_second': 0.156, 'total_flos': 0.0, 'train_loss': 0.0012243206474032604})

In [7]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.69it/s, est. speed input: 99.80 toks/s, output: 43.15 toks/s]


'There are no letters \'r\' in the word "strawberry."'

In [8]:
model.save_lora("grpo_saved_lora")

In [20]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.62s/it, est. speed input: 16.42 toks/s, output: 80.97 toks/s]


'<reasoning>\nTo determine the number of \'r\'s in the word "strawberry," I will go through the word character by character and count each occurrence of the letter \'r\'. \n\n1. The first character is \'s\', not an \'r\'.\n2. The second character is \'t\', not an \'r\'.\n3. The third character is \'r\', an \'r\'.\n4. The fourth character is \'w\', not an \'r\'.\n5. The fifth character is \'a\', not an \'r\'.\n6. The sixth character is \'r\', an \'r\'.\n7. The seventh character is \'r\', an \'r\'.\n8. The eighth character is \'b\', not an \'r\'.\n9. The ninth character is \'b\', not an \'r\'.\n\nAfter analyzing each character, I find three occurrences of the letter \'r\' in the word "strawberry."\n</reasoning>\n<answer>\nThere are 3 \'r\'s in the word "strawberry." </answer>'

In [18]:
prompt = """You are evaluating a profile against both base requirements and enrichment data.

    USER REQUIREMENTS:
Find buyer intent score for property and casualty insurance firms, companies, or agencies that strictly follow the below criteria:
- Headquarters in a 30-mile radius of Fort Myers, Florida (Headquarters location is the strict criteria for location matching),
-Headcount with 11-50 employees, and
- an annual revenue between USD 2.5M to USD 20M.

SPECIFIC CRITERIA TO CHECK:
Base Requirements: {'location_match': 'critical', 'industry_match': 'critical', 'target_market_fit': 'high'}
Required Enrichments: [{'type': 'company_headcount', 'importance': 'critical'}, {'type': 'revenue_estimation', 'importance': 'critical'}, {'type': 'industry_classification', 'importance': 'high'}, {'type': 'profile_activity', 'importance': 'normal'}]

SCORING RULES:

1. For each requirement specified in user requirements:
   - If requirement has a numerical threshold (e.g., revenue > X):
     * Score 5/5 if value found EXCEEDS threshold
     * Score 0/5 if value found is BELOW threshold
     * Score 0/5 if no value found
   - If requirement is binary (e.g., must be in location X):
     * Score 5/5 if exact match
     * Score 0/5 if no match
  - If requirement has an compund inequality (e.g., revenue > X and revenue < Y or  min < X  < max):
    * Score 5/5 if value found is within the range
    * Score 0/5 if value found is outside the range
    * Score 0/5 if no value found

2. Missing Data Rules:
   - If data is found and meets requirement:
     * Category MUST NOT appear in missing_impact
   - If no data found for required field:
     * MUST appear in missing_impact with appropriate importance level

3. Confidence Scores:
   - Base on quality and reliability of evidence found
   - Must justify any confidence score given


    CATEGORIES TO EVALUATE:

    1. Base Requirements (Must always be evaluated):
    - location_match: Geographic location alignment
    - industry_match: Industry sector alignment
    - target_market_fit: Match with target market segment

    2. Enrichment Categories (Based on available data):
    - competitor_analysis
    - news_summary
    - industry_classification
    - hiring_analysis
    - funding_intelligence
    - revenue_estimation
    - profile_activity
    - company_headcount
    PROFILE TO EVALUATE:
    {'company_id': 'iron-ridge-insurance', 'name': 'Iron Ridge Insurance Services, a Member of the Hilb Group', 'description': "Iron Ridge Insurance Services, a member of the Hilb Group is a property and casualty insurance agency headquartered in Fort Myers, FL. Our primary focus is commercial insurance. We provide expertise in professional liability, management liability, data breach liability, general liability, commercial property, workers' compensation, and commercial auto insurance. While we represent clients in a large variety of businesses, we have particular expertise in managing risk for Architects and Engineers, Physicians and Medical Practices, Fitness Centers, Artisan Contractors and Main Street businesses.", 'website': 'http://www.ironridgeus.com', 'size': '11-50 employees', 'industry': ['Insurance'], 'other_company_locations': [], 'followers': 0, 'employees': [{'name': 'Lisa Scholder', 'linkedin_profile_url': 'https://www.linkedin.com/in/lisa-scholder-47079519?trk=org-employees'}, {'name': 'Gary Roth', 'linkedin_profile_url': 'https://www.linkedin.com/in/gary-roth-bab3b146?trk=org-employees'}, {'name': 'Paula LaLande', 'linkedin_profile_url': 'https://www.linkedin.com/in/paula-lalande-91759349?trk=org-employees'}, {'name': 'Cyndi Doragh', 'linkedin_profile_url': 'https://www.linkedin.com/in/cyndi-doragh-18127490?trk=org-employees'}], 'founded': 2011, 'headquarters': 'Fort Myers, Florida', 'specialties': ['Liability Coverage', 'Risk Management', 'Health Care Risk', 'Litigation-Free Practice', 'Professional Liability', 'Commercial Property', 'General Liability', "Workers' Compensation", 'Management Liability', 'Data Breach Liability', 'Inland Marine', 'Flood', 'Umbrella or Excess Insurance', 'Managed Care Liability', 'Key Person Life', 'Disability', 'Commercial Liability', 'Bonds'], 'image': 'https://media.licdn.com/dms/image/v2/C4E0BAQFD3XTBP2jE3g/company-logo_100_100/company-logo_100_100/0/1630569456947/iron_ridge_insurance_logo?e=1747872000&v=beta&t=unBe9DWSm5-OYItOm8B4wIkeXR-5GiQgWrDI91biyVQ', 'country_code': '', 'profile_url': 'https://www.linkedin.com/company/iron-ridge-insurance', 'agent_id': '4cb65ef2-fed1-4703-8f90-5cb20227433a'}

    RESPONDER REQUIREMENTS:
    {'product_description': 'OneDigital provides a comprehensive suite of insurance, financial services, and HR consulting solutions designed to support both employers and individuals in achieving health, success, and financial security. Their personalized, tech-enabled approaches cater to modern work-life experiences and are backed by a robust national presence with over 200 offices across the United States.', 'target_market': 'Insurance, Financial Services, Human Resources, Corporate Businesses, Healthcare', 'additional_context': {'target_decision_makers': '', 'target_locations': 'Fort Myers, Florida', 'target_industries': 'Insurance, Financial Services, Human Resources, Corporate Businesses, Healthcare'}}

    ENRICHMENT DATA STATUS:
    Available: [{'type': 'company_headcount', 'data': {'company_size': '11-50 employees', 'is_available': True}}, {'type': 'revenue_estimation', 'data': {'revenue_details': '$5M-$10M', 'revenue_links': ['https://ironridgeinsurance.com', 'https://pressrelease.com'], 'is_available': True}}, {'type': 'industry_classification', 'data': {'company_industry': ['Insurance'], 'is_available': True}}]
    Missing: [{'type': 'profile_activity', 'importance': 'normal'}]

    Return Python dictionary with exact format shown in example:
    <example_output>
    {
        "scores": {
            "location_match": 5,
            "industry_match": 4,
            "target_market_fit": 3,
            "competitor_analysis": 2,
            "news_summary": 3,
            "industry_classification": 4,
            "hiring_analysis": 3,
            "funding_intelligence": 0,
            "revenue_estimation": 0,
            "profile_activity": 3,
            "company_headcount": 3
        },
        "justifications": {
            "location_match": "Based in target location Bangalore, India",
            "industry_match": "Strong alignment with AI/ML industry focus",
            "target_market_fit": "Matches enterprise customer profile",
            "competitor_analysis": "Limited competitive insights available",
            "revenue_estimation": "No revenue data available",
            "profile_activity": "Recent activity of the user on linkedin. Posts, comments, likes etc indicating the engagement level and topics written of the user.",
            "company_headcount": "Whether the headcount details of the company is within the user requirements or not"
        },
        "confidence": {
            "location_match": 1.0,
            "industry_match": 0.8,
            "target_market_fit": 0.7,
            "competitor_analysis": 0.4,
            "revenue_estimation": 0.0,
            "profile_activity": 0.9,
            "company_headcount": 0.9
        },
        "missing_impact": {
            "revenue_estimation": {
                "importance": "critical",
                "impact": "Cannot verify revenue requirements as it is not available"
            },
            "funding_intelligence": {
                "importance": "normal",
                "impact": "Limited understanding of company stage as it is not available"
            }
        }
    }
    </example_output>
    Note:
    - Base requirements must always have scores, even if low confidence
    - Enrichment categories can have 0 scores if data missing
    - Each 0 score must have corresponding missing_impact entry
    - Justifications must be specific to the enrichment data and should not have duplicate entries.
    - Justifications should always be provided for available required enrichments.
    Always Return Output in Strict JSON format with all the keys mentioned above.
    Always Return a single valid JSON object.
    Do not add any other text or comments.
    Do not add any Markdown or ```json"""

In [22]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : prompt},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 16384,
)
input_tokens = tokenizer(text, return_tensors="pt").input_ids
input_token_length = input_tokens.shape[1]
print(f"Input token length: {input_token_length}")
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)

output[0].outputs[0].text

Input token length: 2056


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.39s/it, est. speed input: 321.91 toks/s, output: 85.64 toks/s]


'<reasoning>\nThe profile provided has a headquarters in Fort Myers, Florida, which matches the user\'s criteria for location match with a score of 5/5. The industry classification matches the required \'Insurance\', so it scores 4/5. However, the target market fit is not explicitly mentioned in the provided profile, so it scores 3/5. Since the enrichment data is missing revenue estimation and funding intelligence, these categories will score 0/5 each. The company headcount of 11-50 employees matches the user\'s criteria, scoring 3/5. The missing impact for revenue estimation will be included as a note, and funding intelligence will not be scored as it is not available.\n</reasoning>\n<answer>\n{\n    "scores": {\n        "location_match": 5,\n        "industry_match": 4,\n        "target_market_fit": 3,\n        "competitor_analysis": 2,\n        "news_summary": 3,\n        "industry_classification": 4,\n        "hiring_analysis": 3,\n        "funding_intelligence": 0,\n        "reven

In [23]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/averma12", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [24]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/averma12", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )